In [ ]:
# Import pytorch 
import torch
from torch import nn

In [ ]:
class Dataset:

    classes = {'amur leopard': 0, 'amur tiger': 1, 'birds': 2, 'black bear': 3, 'brown bear': 4, 'dog': 5, 'roe deer': 6, 'sika deer': 7, 'wild boar': 8, 'people': 9}

    def __init__(self, path):
        self.path = path
        self.classes = classes if classes else Dataset.classes